In [27]:
import statistics
from statistics import mean
import pandas as pd
import spacy

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch.nn.functional as F
from transformers import pipeline

In [ ]:
#langauge detection
from langdetect import detect
from langdetect import detect_langs

After having files generated with the speech-to-text program I make them all into one csv file 
including the TikTok ID and user ID.

In [31]:
df = pd.read_csv('speech-to-text-generated.csv').drop("Unnamed: 0", axis=1)
df = df.dropna()

Code sinpets from:

https://rafalrolczynski.com/2021/03/07/aspect-based-sentiment-analysis/
https://github.com/ScalaConsultants/Aspect-Based-Sentiment-Analysis
https://towardsdatascience.com/nlp-project-with-augmentation-attacks-aspect-based-sentiment-analysis-3342510c90e7

https://medium.com/nlplanet/quick-intro-to-aspect-based-sentiment-analysis-c8888a09eda7

In [35]:
# Load Aspect-Based Sentiment Analysis model
absa_tokenizer = AutoTokenizer.from_pretrained("yangheng/deberta-v3-base-absa-v1.1")
absa_model = AutoModelForSequenceClassification \
  .from_pretrained("yangheng/deberta-v3-base-absa-v1.1")

# Load a traditional Sentiment Analysis model
sentiment_model_path = "cardiffnlp/twitter-xlm-roberta-base-sentiment"
sentiment_model = pipeline("sentiment-analysis", model=sentiment_model_path,
                          tokenizer=sentiment_model_path)

c:\Users\Kinga\anaconda3\lib\site-packages\transformers\convert_slow_tokenizer.py:446: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [38]:
df_prob = pd.DataFrame(columns=["negative", "neutral", "positive", "text","id"])

df['col_isalphanumeric'] = df["col"].str.isspace()
df = df[df['col_isalphanumeric'] ==False]

In [1]:
# This Sentiment Analysis works best for English so I filter out other languges.

In [40]:
#detect language
lang_list = []
i = 0
for text in df["col"]:
    try: 
        lang_list.append(detect(text))
    except:
        lang_list.append("Nan")

df["language"] = lang_list 

In [42]:
df = df[df['language'] =='en']

In [ ]:
def string_aspect_chunks(text,aspect):
    chunk_list = []
    doc = nlp(text)
    for sent in doc.sents:
        if sent.text.find(aspect)!= -1:
            chunk_list.append(sent.text)
    return chunk_list

In [44]:
def make_sentiment_aspect_df(aspect, df):
    df_prob = pd.DataFrame(columns=["negative", "neutral", "positive", 
                                    "text", "aspect", "video_id", "vodeo_user_id"])

    positive_sentiment = []
    negative_sentiment = []
    neutral_sentiment = []
    sentence_list = []
    

    df = df[df["col"].str.find(aspect) != -1]
    
    for sentence_one in df["col"]:
        text = string_aspect_chunks(sentence_one,aspect) 
        positive = []
        negative = []
        neutral = [] 
        score = [] 
        for sentence in text:
            sentence_list.append(sentence)
            inputs = absa_tokenizer(f"[CLS] {sentence} [SEP] {aspect} [SEP]", return_tensors="pt")
            outputs = absa_model(**inputs)
            probs = F.softmax(outputs.logits, dim=1)
            probs = probs.detach().numpy()[0]
            for prob, label in zip(probs, ["negative", "neutral", "positive"]):
                if label == "negative":
                    negative.append(prob)
                if label == "neutral":
                    neutral.append(prob)
                if label == "positive":
                    positive.append(prob)
                
        positive_sentiment.append(mean(positive))
        negative_sentiment.append(mean(negative))
        neutral_sentiment.append(mean(neutral))
                                 
    df_prob["video_id"] = df["video_id"]
    df_prob["negative"] = negative_sentiment
    df_prob["positive"] = positive_sentiment
    df_prob["neutral"] = neutral_sentiment
    df_prob["text"] = df["col"]
    df_prob["vodeo_user_id"] = df["vodeo_user_id"]
    


    df_prob["aspect"]= aspect
    return df_prob



In [45]:
#my study was on Russian Propaganda but you can create your own list or import it 
aspect_list = ["Volodymyr", "Zelenskyy", "Ukrain","Russia", "Vladimir", "Putin",
                "nato", "war", "military", "politi", "west", "east", "Poland",
                "Dutch", "Polish", "Netherlands", "US", "America", "United States", "EU", "Europe"]

In [47]:
final_df = pd.concat((make_sentiment_aspect_df(aspect, df) for aspect in aspect_list ), ignore_index=True)